In [ ]:
import csv
import random
import keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import seaborn as sns
from keras.layers import Input,Conv2D,LSTM
from keras.layers import Reshape
from keras.callbacks import EarlyStopping
import numpy as np

import pandas as pd
from os import getcwd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import save_model, load_model

#import keras_tuner
import tensorflow as tf

from numpy.random import seed
from numpy.random import randn
from scipy.stats import ttest_ind


from scipy import stats


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
    print('and then re-execute this cell.')
else:
    print(gpu_info)

Tue Aug 17 19:11:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU   GI   CI        PID   Type   Process name                  GPU Memory |
|        ID   ID                                                   Usage      |
|=============================================================================|
|  No running processes found                                                 |
+-----------------------------------------------------------------------------+

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
    print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
    print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
    print('re-execute this cell.')
else:
    print('You are using a high-RAM runtime!')

In [ ]:
!pip install -q -U keras-tuner

In [ ]:
import keras_tuner as kt

In [ ]:
def build_model_train(ak,bk,ck):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(191,1)))
    for ix in range(0,ak):
        model.add(tf.keras.layers.LSTM(units=bk,
                            activation = 'tanh',
                            recurrent_activation = 'sigmoid',
                            return_sequences=True,
                           recurrent_dropout = 0))

    model.add(tf.keras.layers.LSTM(units=bk,  activation = 'tanh'))
    model.add(tf.keras.layers.Dense(1))

    model.summary()

    model.compile(
        optimizer=keras.optimizers.RMSprop(
        learning_rate=ck*2),
        loss='mae',
        metrics=['mae'])
    return model

In [ ]:
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(191,1)))

    for i in range(hp.Int('num_layers', 2, 6)):
            model.add(
                     tf.keras.layers.LSTM(units=hp.Int('units_',
                                           128, 2048, 64),
                                activation = 'tanh',
                                recurrent_activation = 'sigmoid',
                                return_sequences=True,
                               recurrent_dropout = 0))
    model.add(tf.keras.layers.LSTM(units=hp.Int('units_',128, 2048, 64),  activation = 'tanh'))
    model.add(tf.keras.layers.Dense(1))

    
    model.compile(
            optimizer=keras.optimizers.RMSprop(
                hp.Choice('learning_rate', [ 1e-3, 1e-4,1e-5,1e-6])),
            loss='mae',
            metrics=['mae'])
    return model

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
def create_dataset(X, time_steps=1):
    Xs = []
    for i in range(time_steps,len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
    return np.array(Xs)
df=pd.read_csv('/content/drive/MyDrive/ETHUSDT15min.csv')
len(df)
df['index'] = np.arange(len(df))
df['index']=df['index']+1
train_size = len(df[:4*24*365+4*24*90*2])
test_size = len(df) - train_size

train, test = df.price_close.iloc[:train_size], df.price_close.iloc[train_size:len(df)]
print(train.shape, test.shape)

train=pd.DataFrame(train)
test=pd.DataFrame(test)

time_steps = 4*24*2

X_train= create_dataset(train, time_steps)
X_test= create_dataset(test,time_steps)

print(X_train.shape)

y_train = X_train[:,-1]
X_train = X_train[:,:-1]

y_test = X_test[:,-1]
X_test = X_test[:,:-1]

X_train.shape

In [ ]:
%%time

from kerastuner import BayesianOptimization

tuner_bo = kt.BayesianOptimization(
            build_model,
            objective='mae',
            max_trials=6,
            seed=42,
            executions_per_trial=1,
            overwrite=True
        )
tuner_bo.search(X_train, y_train,  epochs=1, validation_split=0.2, verbose=1)

best_model = tuner_bo.get_best_models(num_models=1)[0]

Trial 6 Complete [00h 09m 53s]
mae: 38.366519927978516

Best mae So Far: 30.214330673217773
Total elapsed time: 01h 05m 15s
INFO:tensorflow:Oracle triggered exit
CPU times: user 48min 28s, sys: 1min 10s, total: 49min 38s
Wall time: 1h 5min 40s

In [ ]:
tuner_bo.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name='mae', direction='min')
Trial summary
Hyperparameters:
num_layers: 2
units_: 2048
learning_rate: 0.001
Score: 30.214330673217773
Trial summary
Hyperparameters:
num_layers: 2
units_: 576
learning_rate: 0.001
Score: 35.680477142333984
Trial summary
Hyperparameters:
num_layers: 4
units_: 1344
learning_rate: 0.001
Score: 38.366519927978516
Trial summary
Hyperparameters:
num_layers: 6
units_: 2048
learning_rate: 0.001
Score: 48.298255920410156
Trial summary
Hyperparameters:
num_layers: 3
units_: 1856
learning_rate: 1e-05
Score: 155.6876678466797
Trial summary
Hyperparameters:
num_layers: 5
units_: 192
learning_rate: 1e-06
Score: 220.03173828125


In [ ]:
hps = tuner_bo.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values 
a=hps['num_layers']
b=hps['units_']
c=hps['learning_rate']

In [ ]:
a=3
b=2048
c=6.250000073038109e-09/2

In [ ]:
tf.keras.backend.clear_session()
model1=build_model_train(a,b,c)
model1.load_weights('/content/drive/MyDrive/checkpoint2/2period_12-0.58.hdf5')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
===============================================================
lstm (LSTM)                  (None, 191, 2048)         16793600  
_________________________________________________________________
lstm_1 (LSTM)                (None, 191, 2048)         33562624  
_________________________________________________________________
lstm_2 (LSTM)                (None, 191, 2048)         33562624  
_________________________________________________________________
lstm_3 (LSTM)                (None, 2048)              33562624  
_________________________________________________________________
dense (Dense)                (None, 1)                 2049      
===============================================================
Total params: 117,483,521
Trainable params: 117,483,521
Non-trainable params: 0
_________________________________________________________________

In [ ]:
es=tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4,min_delta=0.0001, mode='min',restore_best_weights=True)
n=tf.keras.callbacks.TerminateOnNaN()
rp=tf.keras.callbacks.ReduceLROnPlateau(monitor='loss',mode='min', factor=0.3, patience=1, verbose=2, min_delta=0.0001, cooldown=0, min_lr=0)
checkpoint_filepath = '/content/drive/MyDrive/checkpoint7/retrain-{epoch:02d}-{loss:.2f}.hdf5'
mcc = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='loss',
    mode='min',
    save_best_only=True)

In [ ]:
history = model1.fit(X_train,y_train,
            batch_size=320,
            verbose=2,
            epochs=200,
            callbacks=[es,n,rp])

In [ ]:
plt.plot(history.history['loss'],'b')
plt.show()

In [ ]:
with open("/content/drive/MyDrive/checkpoint2/diff_eth_2.csv") as file_name:
    diff = np.loadtxt(file_name, delimiter=",")

In [ ]:
diff

In [ ]:
diff_eth_train_percent=[]
k=0
for j in range(0,len(X_train)):
    X_train_pred= model1.predict(np.expand_dims(X_train[j],axis=0))
    print (k)
    k+=1
    dif=(X_train_pred-y_train[j])/y_train[j]
    diff_eth_train_percent=np.append(diff_eth_train_percent,dif)

In [ ]:
np.savetxt("/content/drive/MyDrive/checkpoint2/diff_eth_train_percent.csv", diff_eth_train_percent, delimiter=",")

In [ ]:
from keras.callbacks import Callback

class stopAtLossValue(Callback):

        def on_batch_end(self, batch, logs={}):
            THR = 0.6 #Assign THR with the value at which you want to stop training.
            if logs.get('loss') <= THR:
                 self.model.stop_training = True
sal=stopAtLossValue()

es=tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4,min_delta=0.0001, mode='min',restore_best_weights=True)
n=tf.keras.callbacks.TerminateOnNaN()
rp=tf.keras.callbacks.ReduceLROnPlateau(monitor='loss',mode='min', factor=0.3, patience=1, verbose=2, min_delta=0.0001, cooldown=0, min_lr=0)

In [ ]:
with open("/content/drive/MyDrive/checkpoint2/diff_eth_2.csv") as file_name:
    diff = np.loadtxt(file_name, delimiter=",")
    
diff_test_n=diff
import statistics
from scipy.stats import norm
eh=225
var_ep=np.linspace(225, 225*2, num=10)

mean=stats.describe(diff_test_n)[2]
var=stats.describe(diff_test_n)[3]
up_level=mean+3*var
down_level=mean-3*var


y_test_pred_n=[]
y_test_real_n=[]
an_n=[]
an_in=[]
p_val=[]
k1=[]


for i in range (0,int(len(X_test)/191.)):
    print (i)

    X_test_n=X_test[i*191:(i+1)*191]
    y_test_n=y_test[i*191:(i+1)*191]
    print (X_test_n.shape)
    
    for j in range(0,len(X_test_n)):
    
    
    X_test_pred= model1.predict(np.expand_dims(X_test_n[j],axis=0))

 
    d=y_test_n[j]-X_test_pred
    diff_test_n=np.append(diff_test_n,d)
    mean=stats.describe(diff_test_n)[2]
    var=stats.describe(diff_test_n)[3]
    up_level=mean+3*var
    down_level=mean-3*var

    



    # compare samples
    stat, p = ttest_ind(diff, diff_test_n)

    alpha = 0.4

    if p > alpha:
        
        print('Same distributions')
        p_val=np.append(p_val,p)

        y_test_pred_n=np.append(y_test_pred_n,X_test_pred)
        y_test_real_n=np.append(y_test_real_n,y_test_n[j])

        if d<down_level:
            an_in=np.append(an_in,1)
        elif d>up_level:
            an_in=np.append(an_in,-1)
        else:
            an_in=np.append(an_in,0)

        continue

    else:
        
        print('Different distributions')

        p_val=np.append(p_val,p)
        X_test_train_batch=X_test[i*191:(i+1)*191][0:j]
        y_test_train_batch=y_test[i*191:(i+1)*191][0:j]


        ver_d=norm.cdf(d, loc=mean, scale=var)
        print ('mean',mean, 'var',var, abs(ver_d))


        eh=var_ep[int(8*abs((mean-ver_d)))]
        
        print (eh)




        
          
        #train on batch
        model1.compile(
                            optimizer=keras.optimizers.RMSprop(
                            learning_rate=c,momentum=0.35),
                            loss='mae',
                            metrics=['mae'])

        history=model1.fit(X_test_train_batch,y_test_train_batch,
                            batch_size=32,
                            verbose=1,
                            epochs=int(eh),
                            callbacks=[es,n,rp,sal])
      
        X_test_pred= model1.predict(np.expand_dims(X_test_n[j],axis=0))


        d=y_test_n[j]-X_test_pred
        diff_test_n=np.append(diff_test_n,d)
        diff=diff_test_n

        y_test_pred_n=np.append(y_test_pred_n,X_test_pred)
        y_test_real_n=np.append(y_test_real_n,y_test_n[j])

        mean=stats.describe(diff_test_n)[2]
        var=stats.describe(diff_test_n)[3]
        up_level=mean+3*var
        down_level=mean-3*var


        if d<down_level:
            an_in=np.append(an_in,1)
        elif d>up_level:
            an_in=np.append(an_in,-1)
        else:
            an_in=np.append(an_in,0)



    print (len(y_test_pred_n),len(y_test_real_n),len(an_in),len(p_val))      
    df = pd.DataFrame({"pred" : y_test_pred_n, "real" : y_test_real_n, "an_diff_in" : an_in, "p_val" : p_val})
    df.to_csv("/content/drive/MyDrive/checkpoint2/train_on_batch_foofoofoo_2_max_error.csv")
    


In [ ]:
df = pd.DataFrame({"pred" : y_test_pred_n, "real" : y_test_real_n, "an_diff_in" : an_in, "p_val" : p_val})
df.to_csv("/content/drive/MyDrive/checkpoint2/train_on_batch_foofoofoo_2.csv")